# **StarLift Airlines Trips**
As an introductory problem to data and network optimization using Python, we'll go over this problem with a made-up airline company trying to find the best setup for their flights. The original dataset can be found here: https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat. Note that how we use it here is (most likely) out of its original context.

In [ ]:
# Import statements
import pandas as pd # Pandas library that allows us to use dataframes
import matplotlib.pyplot as plt # This library allows us to visualize our data
import math
import numpy as np

# Get our custom functions
import sys # Allows us to import functions from a different directory
sys.path.append('./Course Resources/') # Add the directory that has our functions file
import functions as fun # Import our functions from the functions.py file. Go check it out in the Course Resources folder!

## **StarLift Airlines Analysis - Static**<br>
The goal of this analysis is to analyze the structure of a commercial airline's flight system. Let's call the company StarLift Airlines and we'll say they fly all over the world. StarLift has several airports they fly to and many different routes between airports, and they want to find what the most profitable setup is at different price points and airport fees. They want to find out if they should remove airports and, if so, how many they should remove. Your task is to do an analysis of the trip data provided to find the number of airports they fly to/from, the most used airports, and create a model showing the relationship between potential profits and airports removed.

In [ ]:
# Create our dataframe from the csv file
df = pd.read_csv('./Data/airplaneRoutes.csv') # Read in the data from the csv file
df.columns = ['source_airport', 'dest_airport'] # Traditional python naming conventions for columns
df

In [ ]:
# Initial assumptions: variables we will use to change projections
PROFIT_PER_TRIP = 10000
FEE_PER_AIRPORT = 200000

# Store the number of trips in a variable that we'll use in our projections
N_TRIPS = df.shape[0]

# Output to make sure we got the right numbers
print(f'Charge per trip: ${PROFIT_PER_TRIP}\nFee per airport: ${FEE_PER_AIRPORT}\nNumber of trips: {N_TRIPS}')

In [ ]:
# Separate the source airports and destination airports into separate dataframes. Get unique values, sort them, and reset the index
unique_source_airports = df[['source_airport']].drop_duplicates().sort_values(by='source_airport').reset_index(drop=True)
unique_dest_airports = df[['dest_airport']].drop_duplicates().sort_values(by='dest_airport').reset_index(drop=True)

fun.compare_dataframes(unique_source_airports, unique_dest_airports) # Compare the dataframes

# Check if all of the source airports are also in the destination airports
unique_dest_airports['dest_airport'].isin(unique_source_airports['source_airport']).value_counts()

# Get the values that are not in source_airports and append them to the end of the dataframe
missing_airports = unique_dest_airports[~unique_dest_airports['dest_airport'].isin(unique_source_airports['source_airport'])]

# Rename the columns to be identical
missing_airports.columns = ['source_airport']

# Combine the dataframes to get all the unique values
unique_airports = pd.concat([unique_source_airports, missing_airports])
unique_airports.columns = ['airportId']
unique_airports

In [ ]:
# Create an array of unique airports in our dataset. Store the number of airports
N_AIRPORTS = unique_airports.shape[0]

# Output the number of airports
print(f'Number of airports: {N_AIRPORTS}')

In [ ]:
# Get the counts of each unique airport from the original dataframe and sort by airportId
source_airports = pd.DataFrame(df['source_airport'].value_counts().sort_index())
dest_airports = pd.DataFrame(df['dest_airport'].value_counts().sort_index())

# Combine the two dataframes to the unique_airports dataframe to display counts
unique_airports = pd.merge(unique_airports, source_airports, left_on='airportId', right_index=True, how='left')
unique_airports = pd.merge(unique_airports, dest_airports, left_on='airportId', right_index=True, how='left')

# Rename the columns to be more descriptive
unique_airports.columns = ['airportId', 'source_flights', 'dest_flights']

# Fill in the NaN values with 0
unique_airports.fillna(0, inplace=True)

# Change the columns to integers
unique_airports['source_flights'] = unique_airports['source_flights'].astype(int)
unique_airports['dest_flights'] = unique_airports['dest_flights'].astype(int)

# Create column to keep track of double counts
unique_airports['double_counts'] = 0

# Make the index the airportId
unique_airports.set_index('airportId', inplace=True)

# Output the dataframe
unique_airports

In [ ]:
# Check if there are any trips where the source and destination are the same
print(df[df['source_airport'] == df['dest_airport']])
# There is one trip where the source and destination are the same

''' INSTEAD OF RUNNING THE BELOW CODE...
# Use loop to count double counted trips for each airport
for index, row in source_airports.iterrows():
    airport_id = row.iloc[0] # Get the airport id
    print(airport_id)
    # Count double counted trips where source airport is equal to dest airport
    unique_airports.loc[airport_id, 'double_counts'] = ((df['source_airport'] == airport_id) & (df['dest_airport'] == airport_id)).sum()

# Add start and ends and subtract double.counts to get the total number of times each airport was involved in a trip
unique_airports['total'] = unique_airports['source_flights'] + unique_airports['dest_flights'] - unique_airports['double_counts'] # Add the two columns together

# Output
print("start + end - double_counts = total")

# Sort the rows by 'total' from least to greatest
unique_airports = unique_airports.sort_values(by='total')

'''

In [ ]:
# ...Let's just add one to PKN's double counts
unique_airports.loc[unique_airports.index == 'PKN', 'double_counts'] = 1
unique_airports['total'] = unique_airports['source_flights'] + unique_airports['dest_flights'] - unique_airports['double_counts'] # Add the two columns together
unique_airports = unique_airports.sort_values(by='total')

unique_airports.loc['PKN']

In [ ]:
# Initialize some variables to use in the loop
airports_removed = 0
df_copy = df.copy() # Create a copy of the dataframe to work with. This will be empty by the end of the loop

# Create a dataframe to store the profits for the number of airports removed. Set index equal to the number of airports removed
profitsStatic = pd.DataFrame(index=range(N_AIRPORTS + 1), columns=['profit']).rename_axis('airports_removed')

# Set the first profit to the profit with no airports removed
profitsStatic.at[0, 'profit'] = PROFIT_PER_TRIP * N_TRIPS - FEE_PER_AIRPORT * N_AIRPORTS
profitsStatic.at[0, 'airport_removed'] = "--"

# Iterate through each airport, removing one each iteration, to compare profits and find the max from the 
# formula profits = (PROFIT_PER_TRIP * trips_remaining) - FEE_PER_AIRPORT * airports_remaining
for index, row in unique_airports.iterrows():
    airport_id = index # Get the airport id
    airports_removed += 1 # Increment the number of airports removed
    airports_remaining = N_AIRPORTS - airports_removed # Calculate the number of airports remaining
    
    # Find all instances of the airport in the dataframe, as the source or dest airport, and remove them
    df_copy = df_copy[(df_copy['source_airport'] != airport_id) & (df_copy['dest_airport'] != airport_id)]
    trips_remaining = df_copy.shape[0]
    
    # Calculate the profit for the number of airports removed
    airport_profit = (PROFIT_PER_TRIP * trips_remaining) - (FEE_PER_AIRPORT * airports_remaining)
    
    # Store the profit in the dataframe
    profitsStatic.at[airports_removed, 'profit'] = airport_profit
    profitsStatic.at[airports_removed, 'airport_removed'] = airport_id # Store the airport that was removed
    
# Output the max profit and the airport that was removed
max_profit = profitsStatic['profit'].max() # Find the max profit
max_profit_index = profitsStatic['profit'].idxmax() # Find the index of the max profit
print(f'Max profit of ${max_profit} at {max_profit_index} airports removed from the system')

profitsStatic

In [ ]:
# Create a plot of the profits
plt.plot(profitsStatic.index, profitsStatic['profit'])
plt.xlabel('Airports Removed')
plt.ylabel('Profit')
plt.title('Profit vs. Airports Removed')
plt.show()

## **StarLift Airlines Analysis - Dynamic Update**
In the static example, the counts for each airport were taken and sorted once and that was our final list for removal. To get a more accurate count, though, we want to get the counts for each airport every time we remove an airport since then the counts might change slightly and our order may be different.

In [ ]:
# Create our dataframe from the csv file
df = pd.read_csv('./Data/airplaneRoutes.csv') # Read in the data from the csv file
df.columns = ['source_airport', 'dest_airport'] # Traditional python naming conventions for columns

# Initial assumptions
PROFIT_PER_TRIP = 10000
FEE_PER_AIRPORT = 200000
N_TRIPS = df.shape[0]

df.head()

In [ ]:
# Static function for just this problem
def get_airport_counts (df, double_counts=True, ascending=True):
    import pandas as pd
    # set the index to be the unique values of the source_airport
    counts = df['source_airport'].unique()
    
    # Get the unique values of dest_airport that are not found in source_airport/already in the dataframe
    missing_airports = df['dest_airport'][~df['dest_airport'].isin(counts)].unique()
    
    counts = np.append(counts, missing_airports)
    
    df_counts = pd.DataFrame(index=counts) # Create a new dataframe with the index set to the unique values of source_airport
    
    df_counts['source_airport'] = df['source_airport'].value_counts() # set the first row to the source_airport counts
    df_counts['dest_airport'] = df['dest_airport'].value_counts() # set the second row to the dest_airport counts
    
    df_counts.fillna(0, inplace=True) # Fill in the NaN values with 0
    df_counts['source_airport'] = df_counts['source_airport'].astype(int) # convert the columns to integers
    df_counts['dest_airport'] = df_counts['dest_airport'].astype(int)
    
    # Double counts: the only row in our dataset where the source and destination are the same is PKN, and it only occurs once.
    # Check first if PKN exists in our df_counts dataframe
    if double_counts:
        df_counts['double_counts'] = 0 # Create a column to store the double counts
        if 'PKN' in df_counts.index:
            df_counts.loc['PKN', 'double_counts'] = 1
    
    # Get the total number of trips for each airport
    df_counts['total'] = df_counts['source_airport'] + df_counts['dest_airport'] - df_counts['double_counts']
    
    # Sort the dataframe by the total column
    df_counts = df_counts.sort_values(by='total', ascending=ascending)
    
    return df_counts

df_hardcoded = get_airport_counts(df)

In [ ]:
# Dynamic function to get the counts of each unique ID from each column in the original dataframe
def get_counts(df, tableId, col=[], double_counts="none", ascending=True):
    import pandas as pd
    df_counts = pd.DataFrame() # Create an empty dataframe to store the counts
    col_unique_vals = [] # Create an empty list to store the unique values
    
    # GET ALL THE UNIQUE ID'S FOR EVERY COLUMN IN THE DATAFRAME
    for column in df.columns:
        col_unique_vals = df[column].unique()
        
        # If it's not the first column, remove the values that are already in the dataframe
        if column != df.columns[0]:
            # Assign the values that are not in the dataframe to col_unique_vals
            col_unique_vals = [val for val in col_unique_vals if val not in df_counts.iloc[:, 0].values]
            
            # Make a temporary dataframe to store the unique values
            temp_df = pd.DataFrame(col_unique_vals)
        
            # Concatenate temp_df with df_counts
            df_counts = pd.concat([df_counts, temp_df], ignore_index=True)
        else:
            # This is the first column, so create the dataframe based on the unique values in the column
            df_counts = pd.DataFrame(col_unique_vals)
            
        col_unique_vals = [] # Reset the list of unique values
    
    # RENAME THE COLUMNS TO BE MORE DESCRIPTIVE. THIS WILL LATER BE SET TO THE INDEX
    df_counts.columns = [tableId]
    
    # GET THE COUNTS OF EACH UNIQUE ID FROM EACH COLUMN IN THE ORIGINAL DATAFRAME
    i = 1
    for column in df.columns:
        col_counts = pd.DataFrame(df[column].value_counts())
        col_name = f'col_{i}'  # Create custom column name
        col_counts.columns = [col_name]  # Assign custom column name
        df_counts = pd.merge(df_counts, col_counts, left_on=tableId, right_index=True, how='left')
        i += 1

    # SET THE INDEX TO THE ID COLUMN
    df_counts.set_index(tableId, inplace=True)
    
    # DYNAMICALLY ASSIGN COLUMN NAMES. FIRST CHECK IF THE USER PROVIDED A VALID LIST OF COLUMN NAMES. IF NOT, CREATE DEFAULT COLUMN NAMES
    if len(col) < len(df_counts.columns):
        num_missing_cols = len(df_counts.columns) - len(col)
        default_cols = [f'col_{i+1}' for i in range(len(col), len(col) + num_missing_cols)]
        col.extend(default_cols)
    # Create a dictionary to map existing column names to new column names
    column_mapping = {old_col: new_col for old_col, new_col in zip(df_counts.columns, col)}
    df_counts.rename(columns=column_mapping, inplace=True)
    
    # REPLACE NaN VALUES WITH 0 AND CHANGE THE COLUMNS TO INTEGERS
    df_counts.fillna(0, inplace=True)
    for column in df_counts.columns:
        df_counts[column] = df_counts[column].astype(int)
    
    # DOUBLE COUNTING
    if double_counts == "byrow": # If there is at least two of one value in a row, count it as one
        df_counts['double_counts'] = 0 # Create a column to store the double counts
        columnNamesOfOriginalData = df.columns # Get the column names of the original data (used in the loop below)
        
        # Tally the double counts dynamically
        for index in df_counts.index:
            df_counts.at[index, 'double_counts'] = ((df[columnNamesOfOriginalData] == index).sum(axis=1) == len(col)).sum()
            
        # Add together all the rows (except for double_counts), subtracting double counts at the end
        df_counts['total'] = df_counts.drop(columns='double_counts').sum(axis=1) - df_counts['double_counts']
    elif double_counts == "individual": # Count every repeated value individually for each row
        df_counts['total'] = df_counts.sum(axis=1) # need to figure out how to implement this
    else:
        # Otherwise, just add all the rows together
        df_counts['total'] = df_counts.sum(axis=1)
    
    # SORT THE ROWS BY 'total' BASED ON THE ASCENDING PARAMETER
    df_counts = df_counts.sort_values(by='total', ascending=ascending)
        
    return df_counts

df_new = get_counts(df, "airportId", col=['source_flights', 'dest_flights'], double_counts="byrow")
df_new

In [ ]:
fun.compare_dataframes(df_new, df_hardcoded) # Compare the dataframes

# Locate where double counts are greater than 0
print(df_new.loc[df_new['double_counts'] > 0])

# Find the row where PKN is the source_airport and the dest_airport
df.loc[(df['source_airport'] == 'PKN') & (df['dest_airport'] == 'PKN')]

In [ ]:
# Initialize some variables to use in the loop
airports_removed = 0
df_copy = df.copy() # Create a copy of the dataframe to work with. This will be empty by the end of the loop
airport_counts = get_airport_counts(df_copy, double_counts=True) # Initialize airport_counts

N_AIRPORTS = airport_counts.shape[0] # Initialize the number of airports

# Create a dataframe to store the profits for the number of airports removed. Set index equal to the number of airports removed
profitsDynamic = pd.DataFrame(index=range(airport_counts.shape[0] + 1), columns=['profit']).rename_axis('airports_removed')

# Set the first profit to the profit with no airports removed
profitsDynamic.at[0, 'profit'] = PROFIT_PER_TRIP * N_TRIPS - FEE_PER_AIRPORT * N_AIRPORTS
profitsDynamic.at[0, 'airport_removed'] = "--"
    
while airport_counts.shape[0] > 0:
    airport_id = airport_counts.index[0] # Get the airport id
    
    # Find all instances of the airport in the dataframe, as the source or dest airport, and remove them
    df_copy = df_copy[(df_copy['source_airport'] != airport_id) & (df_copy['dest_airport'] != airport_id)]
    trips_remaining = df_copy.shape[0] # Calculate the number of trips remaining
    
    # Recalculate the airport counts
    airport_counts = get_airport_counts(df_copy)

    airports_removed += 1 # Increment the number of airports removed
    airports_remaining = airport_counts.shape[0] # Calculate the number of airports remaining
    
    # Calculate the profit for the number of airports removed
    airport_profit = (PROFIT_PER_TRIP * trips_remaining) - (FEE_PER_AIRPORT * airports_remaining)
    
    # Store the profit in the dataframe
    profitsDynamic.at[airports_removed, 'profit'] = airport_profit
    profitsDynamic.at[airports_removed, 'airport_removed'] = airport_id
    
# Get rid of rows in the dataframe that are all NaN
profitsDynamic = profitsDynamic.dropna(how='all')
    
# Output the max profit and the airport that was removed
max_profit = profitsDynamic['profit'].max() # Find the max profit
max_profit_index = profitsDynamic['profit'].idxmax() # Find the index of the max profit
print(f'Max profit of ${max_profit} at {max_profit_index} airports removed from the system')

profitsDynamic

In [ ]:
# Create a plot of the profits
plt.plot(profitsDynamic.index, profitsDynamic['profit'])
plt.xlabel('Airports Removed')
plt.ylabel('Profit')
plt.title('Profit vs. Airports Removed')
plt.show()

## **Compare Static vs. Dynamic**

In [ ]:
# Combine the dataframes for a side-by-side comparison
profitsCombined = pd.concat([profitsStatic, profitsDynamic], axis=1)
profitsCombined.columns = ['static_profit', 'static_airport_removed', 'dynamic_profit', 'dynamic_airport_removed']
# Add to the dataframe a column for the difference between the static and dynamic profits
profitsCombined['difference_in_profit'] = profitsCombined['dynamic_profit'] - profitsCombined['static_profit']
profitsCombined

In [ ]:
# Compare the max profits between the static and dynamic methods
max_profit_static = profitsStatic['profit'].max()
max_profit_dynamic = profitsDynamic['profit'].max()
max_profit_difference = max_profit_dynamic - max_profit_static
print(f'Max profit difference between static and dynamic methods: ${max_profit_difference}')
print(f'Static: ${max_profit_static}\nDynamic: ${max_profit_dynamic}')

In [ ]:
# Plot the two profits on the same graph, also plot the difference in profits
plt.plot(profitsCombined.index, profitsCombined['static_profit'], label='Static')
plt.plot(profitsCombined.index, profitsCombined['dynamic_profit'], label='Dynamic')
plt.plot(profitsCombined.index, profitsCombined['difference_in_profit'], label='Difference')
plt.xlabel('Airports Removed')
plt.ylabel('Profit')
plt.title('Profit vs. Airports Removed')
plt.legend() # Add a legend to the graph
plt.show()

In [ ]:
# A more clear graph of the difference in profits
plt.plot(profitsCombined.index, profitsCombined['difference_in_profit'], label='Difference')
plt.xlabel('Airports Removed')
plt.ylabel('Difference in Profit')
plt.title('Profit vs. Airports Removed')
plt.show()

# Get the max difference in profits and the number of airports removed
max_diff = profitsCombined['difference_in_profit'].max()
max_diff_index = profitsCombined['difference_in_profit'].idxmax()
print(f'Max difference in profit of ${max_diff} at {max_diff_index} airpots removed from the system')

## **StarLift Airlines Analysis - Random Selection w/ Dynamic Update (Monte Carlo)**
Next, we theorize that even though one airport has more/less trips than another, we don't know how many other airports will be impacted by the removal of the one we intend to remove, and if one airport might have a bigger impact by its removal than another on different airports. Another simulation may be needed to test different methods of airport removal to see if we can get an even higher max profit using some elements of randomness in our process. This next section is just to introduct the idea of random selection even though a Monte Carlo simulation is not intended for such a simple model. Here is a good summary of the methodology: https://aws.amazon.com/what-is/monte-carlo-simulation/

In [ ]:
# Create our dataframe from the csv file
df = pd.read_csv('./Data/airplaneRoutes.csv') # Read in the data from the csv file
df.columns = ['source_airport', 'dest_airport'] # Traditional python naming conventions for columns

# Initial assumptions
PROFIT_PER_TRIP = 10000
FEE_PER_AIRPORT = 200000
N_TRIPS = df.shape[0]
N_AIRPORTS = df['source_airport'].nunique() + df['dest_airport'][~df['dest_airport'].isin(df['source_airport'])].nunique()
M_CARLO_WINDOWS = [int(round(N_AIRPORTS*0.05)), 
                   int(round(N_AIRPORTS*0.1)), 
                   int(round(N_AIRPORTS*0.2))] # 5%, 10%, and 20% of the total number of trips
M_CARLO_RUNS = 10

df.head()

In [ ]:
def MonteCarlo(df, charge, fee, numAirports, numTrips, mCarloWindows, runs, messages='all', profitToBeat=[0, 0]):
    import time
    start_time = time.time() # Get the start time; keep track of runtime

    if profitToBeat[1] == 0:
        profitToBeat[1] = numAirports

    # Store the max we find. Initialize the dataframe
    the_max_df = pd.DataFrame(index=range(numAirports + 1), columns=['profit']).rename_axis('airports_removed')
    the_max_profit = profitToBeat[0]
    the_max_airport_removed = profitToBeat[1]

    # Seed the random number generator with the current time. This will make sure we get different results each time we run the code
    np.random.seed()

    for window in mCarloWindows:
        iterations = math.ceil(numAirports / window) + 1 # Calculate the number of iterations
        if messages in ['simple', 'all']: print(f'Window: {window}, Runs: {runs}, Iterations: {iterations}')
        
        for x in range(runs):
            # Initialize some variables to use in the loop
            airports_removed = 0
            df_copy = df.copy() # Create a copy of the dataframe to work with. This will be empty by the end of the loop
            airport_counts = get_airport_counts(df_copy, double_counts=True) # Initialize airport_counts
            
            # Create a dataframe to store the profits for the number of airports removed. Set index equal to the number of airports removed
            profitsRandomSelection = pd.DataFrame(index=range(airport_counts.shape[0] + 1),\
                                                  columns=['profit']).rename_axis('airports_removed')

            # Set the first profit to the profit with no airports removed
            profitsRandomSelection.at[0, 'profit'] = charge * numTrips - fee * numAirports
            profitsRandomSelection.at[0, 'airport_removed'] = "--"
            
            for i in range(iterations): # Iterate through the number of iterations
                airports_in_window = airport_counts.index[:window] # Get airports in the window
                
                for j in airports_in_window: # Iterate through the window
                    # Get a random airport from the window
                    airport_id = airports_in_window[math.floor(len(airports_in_window) * np.random.random())]
                        
                    # Remove the airport from the airports_in_window
                    airports_in_window = airports_in_window[airports_in_window != airport_id]
                    
                    # Find all instances of the airport in the dataframe, as the source or dest airports, and remove them
                    df_copy = df_copy[(df_copy['source_airport'] != airport_id) & (df_copy['dest_airport'] != airport_id)]
                    trips_remaining = df_copy.shape[0] # Calculate the number of trips remaining
                    
                    # Recalculate the airport counts
                    airport_counts = get_airport_counts(df_copy)
                
                    airports_removed += 1 # Increment the number of airports removed
                    airports_remaining = airport_counts.shape[0] # Calculate the number of airports remaining
                    
                    # Calculate the profit for the number of airports removed
                    airport_profit = (charge * trips_remaining) - (fee * airports_remaining)
                    
                    # Store the profit in the dataframe
                    profitsRandomSelection.at[airports_removed, 'profit'] = airport_profit
                    profitsRandomSelection.at[airports_removed, 'airport_removed'] = airport_id
                
            # Output the max profit and the airport that was removed
            max_profit = profitsRandomSelection['profit'].max() # Find the max profit
            max_profit_index = profitsRandomSelection['profit'].idxmax() # Find the index of the max profit
            
            # Get how far into the runtime we are in minutes and seconds
            elapsed_time = time.time() - start_time 
            elapsed_minutes = int(elapsed_time // 60)
            elapsed_seconds = int(elapsed_time % 60)
            
            if messages in ['all']: print(f'{x + 1}: Max profit of ${max_profit} at {max_profit_index} airports removed ({elapsed_minutes} min {elapsed_seconds} sec)')

            # If the max profit is greater than what we have, record it. If it's the same but at a lower count fo airports removed, record
            if max_profit > the_max_profit:
                if messages in ['simple', 'all']: print(f">> The max profit changed from {the_max_profit} to {max_profit} at {max_profit_index} airports removed")
                the_max_df = profitsRandomSelection.copy()
                the_max_profit = max_profit
                the_max_airport_removed = max_profit_index
            elif max_profit == the_max_profit and max_profit_index < the_max_airport_removed:
                if messages in ['simple', 'all']: print(f"The max profit stayed the same at {max_profit}, but was achieved with {max_profit_index} airports removed")
                the_max_df = profitsRandomSelection.copy()
                the_max_profit = max_profit
                the_max_airport_removed = max_profit_index
                
        if messages in ['simple', 'all']: print() # Add a space between each window
        
    # Output total runtime  
    elapsed_time = time.time() - start_time # Get how far into the runtime we are in minutes and seconds
    elapsed_minutes = int(elapsed_time // 60)
    elapsed_seconds = int(elapsed_time % 60) 
    if messages in ['simple', 'all']: print(f'\nTotal runtime: {elapsed_minutes} min {elapsed_seconds} sec')
    
    # Output a graph of the max profit and the_max_df
    plt.plot(the_max_df.index, the_max_df['profit'])
    plt.xlabel('Airports Removed')
    plt.ylabel('Profit')
    plt.title('Max Profit vs. Airports Removed')
    plt.show()
    
    return the_max_df, the_max_profit, the_max_airport_removed

In [ ]:
# Clear some variables we don't need and won't use again before running to make sure it runs as quickly as possible
del airport_counts, airport_id, airport_profit, airports_remaining, airports_removed, dest_airports, df_copy, df_hardcoded, df_new,\
    index, max_diff, max_diff_index, max_profit, max_profit_difference, max_profit_dynamic, max_profit_index, max_profit_static, \
    missing_airports, profitsDynamic, profitsStatic, row, source_airports, trips_remaining, unique_airports, unique_dest_airports, \
    unique_source_airports

In [ ]:
profitsMonteCarlo, max_profit_mc, max_airports_rem_mc = MonteCarlo(df, PROFIT_PER_TRIP, FEE_PER_AIRPORT, N_AIRPORTS, N_TRIPS, M_CARLO_WINDOWS, M_CARLO_RUNS, messages='all')
profitsMonteCarlo
# Max profit calculated was with dynamic at $359,360,000 with 2567 airports removed

## **Compare Monte Carlo vs. Static and Dynamic**

In [ ]:
# Compare the max profits between the static and dynamic methods
max_profit_dynamic = profitsCombined['dynamic_profit'].max()
max_profit_difference = max_profit_dynamic - max_profit_mc
print(f'Max profit difference between static and dynamic methods: ${max_profit_difference}')
print(f'Monte Carlo: ${max_profit_mc}\nDynamic: ${max_profit_dynamic}')

In [ ]:
# Combine the dataframes for a side-by-side comparison
profitsCombined = pd.concat([profitsCombined, profitsMonteCarlo], axis=1)
profitsCombined.columns = ['static_profit', 'static_airport_removed', 'dynamic_profit', 'dynamic_airport_removed', 'difference_in_profit', 'mc_profit', 'mc_airport_removed']
# Add to the dataframe a column for the difference between the static and dynamic profits
profitsCombined['mc_difference_in_profit'] = profitsCombined['mc_profit'] - profitsCombined['dynamic_profit']
profitsCombined['mc_w_static_difference'] = profitsCombined['mc_profit'] - profitsCombined['static_profit']
profitsCombined

In [ ]:
# A graph of the difference in profits
plt.plot(profitsCombined.index, profitsCombined['mc_difference_in_profit'], label='M-Carlo and Dynamic')
plt.plot(profitsCombined.index, profitsCombined['difference_in_profit'], label='Dynamic and Static')
plt.plot(profitsCombined.index, profitsCombined['mc_w_static_difference'], label='M-Carlo and Static')
plt.xlabel('Airports Removed')
plt.ylabel('Profit')
plt.title('Profit vs. Airports Removed')
plt.legend()
plt.show()

# Get the max difference in profits and the number of airports removed
max_diff = profitsCombined['mc_difference_in_profit'].max()
max_diff_index = profitsCombined['mc_difference_in_profit'].idxmax()
print(f'Max difference in profit of ${max_diff} at {max_diff_index} airports removed from the system')